In [2]:
from transformers import BertTokenizerFast
from ast import literal_eval
import pandas as pd
import re
import tensorflow as tf
import tensorflow_text as text

ImportError: cannot import name 'normalization' from 'tensorflow.python.keras.layers' (C:\Users\Serhan\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\layers\__init__.py)

In [83]:
class Preprocessing:
    def __init__(self):
        return

    def organizeData(self, featuresFile, patientNotesFile, trainFile):
        self.fDF = pd.read_csv(featuresFile)
        self.pnDF = pd.read_csv(patientNotesFile)
        self.tDF = pd.read_csv(trainFile)

        self.tDF['annotation'] = self.tDF['annotation'].apply(literal_eval)
        self.tDF['location'] = self.tDF['location'].apply(literal_eval)

        self.removeORs()
        self.removeSpecialChars()
        
        self.merged = self.tDF.merge(self.pnDF, how="left")
        self.merged = self.merged.merge(self.fDF, how="left")

    def removeORs(self):
        self.fDF['feature_text'] = self.fDF['feature_text'].apply(lambda x: x.lower())
        for i in range(len(self.fDF['feature_text'])):
            self.fDF.at[i, 'feature_text'] = self.fDF['feature_text'][i].replace("-OR-", ";-").replace("-", " ")

    def removeSpecialChars(self):
        self.pnDF['pn_history'] = self.pnDF['pn_history'].apply(lambda x: x.lower())
        for i in range(len(self.pnDF['pn_history'])):
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace("(", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace(")", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace(":", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace(";", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace("-", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace("/", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace("\\", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace("\r\n", "  ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace("\'", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace("\"", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace(",", " ")
            self.pnDF.at[i, 'pn_history'] = self.pnDF['pn_history'][i].replace(".", " ")
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\bzero\b', ' 0  ', self.pnDF['pn_history'][i])
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\bone\b', ' 1 ', self.pnDF['pn_history'][i])
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\btwo\b', ' 2 ', self.pnDF['pn_history'][i])
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\bthree\b', '  3  ', self.pnDF['pn_history'][i])
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\bfour\b', ' 4  ', self.pnDF['pn_history'][i])
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\bfive\b', ' 5  ', self.pnDF['pn_history'][i])
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\bsix\b', ' 6 ', self.pnDF['pn_history'][i])
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\bseven\b', '  7  ', self.pnDF['pn_history'][i])
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\beight\b', '  8  ', self.pnDF['pn_history'][i])
            self.pnDF.at[i, 'pn_history'] = re.sub(r'\bnine\b', ' 9  ', self.pnDF['pn_history'][i])

In [85]:
featuresFile = "..\\CSVs\\features.csv"
patientNotesFile = "..\\CSVs\\patient_notes.csv"
trainFile = "..\\CSVs\\train.csv"

prep = Preprocessing()
prep.organizeData(featuresFile, patientNotesFile, trainFile)

print(prep.fDF['feature_text'][0])
print(prep.pnDF['pn_history'][0])

family history of mi or family history of myocardial infarction
17 year old male  has come to the student health clinic complaining of heart pounding  mr  cleveland s mother has given verbal consent for a history  physical examination  and treatment   began 2 3 months ago sudden intermittent for 2 days lasting 3 4 min  worsening non allev aggrav   associated with dispnea on exersion and rest stressed out about school   reports fe feels like his heart is jumping out of his chest   ros denies chest pain dyaphoresis wt loss chills fever nausea vomiting pedal edeam   pmh non meds  aderol  from a friend  nkda   fh father had mi recently mother has thyroid dz   sh non smoker mariguana 5 6 months ago 3 beers on the weekend  basketball at school   sh no std


In [86]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize(prep.fDF['feature_text'][0], prep.pnDF['pn_history'][0])

print("Total Tokens", len(tokens))
print(tokens)

Total Tokens 162
['family', 'history', 'of', 'mi', 'or', 'family', 'history', 'of', 'my', '##oca', '##rdial', 'in', '##far', '##ction', '17', 'year', 'old', 'male', 'has', 'come', 'to', 'the', 'student', 'health', 'clinic', 'complaining', 'of', 'heart', 'pounding', 'mr', 'cleveland', 's', 'mother', 'has', 'given', 'verbal', 'consent', 'for', 'a', 'history', 'physical', 'examination', 'and', 'treatment', 'began', '2', '3', 'months', 'ago', 'sudden', 'intermittent', 'for', '2', 'days', 'lasting', '3', '4', 'min', 'worse', '##ning', 'non', 'all', '##ev', 'ag', '##gra', '##v', 'associated', 'with', 'di', '##sp', '##nea', 'on', 'ex', '##ers', '##ion', 'and', 'rest', 'stressed', 'out', 'about', 'school', 'reports', 'fe', 'feels', 'like', 'his', 'heart', 'is', 'jumping', 'out', 'of', 'his', 'chest', 'ro', '##s', 'denies', 'chest', 'pain', 'd', '##ya', '##ph', '##ores', '##is', 'w', '##t', 'loss', 'chill', '##s', 'fever', 'nausea', 'vomiting', 'pedal', 'ed', '##ea', '##m', 'pm', '##h', 'non', 

In [92]:
tokenized_list = tokenizer(
        prep.merged.iloc[0].feature_text,
        prep.merged.iloc[0].pn_history,
        truncation=True,
        max_length=1000,
        padding='max_length',
        return_offsets_mapping=True
)
print(prep.merged.iloc[0].pn_history[668:693])

zipped = zip(tokenized_list.sequence_ids(), tokenized_list["offset_mapping"])

idx, (seq_id, offsets) = next(enumerate(zipped))
if not seq_id or seq_id == 0:
    print("Seq ID zero, so level is -1 also")

seq_id = 1 #assume
loc_list = [668, 693]

for idx, (seq_id, offsets)  in enumerate(zip(tokenized_list.sequence_ids(), tokenized_list["offset_mapping"])):
    token_start, token_end = offsets
    for feature_start, feature_end in [loc_list]:
        if token_start >= feature_start and token_end <= feature_end:
            print(f"Word {prep.merged.iloc[0].pn_history[token_start:token_end]}, label: 1")

mom with  thyroid disease
Seq ID zero, so level is -1 also
Word mom, label: 1
Word with, label: 1
Word thyroid, label: 1
Word disease, label: 1
